In [13]:
import cv2
import imutils
vidcap = cv2.VideoCapture("Test Video.mp4") #get image
print (vidcap);

#Function to get frames from the video
def getFrame(sec):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000) 
    hasFrames,image = vidcap.read() #read video if frames avbl
    if hasFrames:
        cv2.imwrite("photos/frame "+str(sec)+" sec.jpg", image)     # save frame as JPG file
        
    return hasFrames

sec = 0
frameRate = 1 #it will capture image in each 1 second
success = getFrame(sec)
while success:
    sec = sec + frameRate
    sec = round(sec, 2)
    success = getFrame(sec)

    

<VideoCapture 0000004E1B9A4B90>


In [15]:
for idx in range(0,40):
    image = cv2.imread("photos/frame "+str(idx)+" sec.jpg")
    #print (image)
    
    image = imutils.resize(image, width=500) #gets it into feasible aspect ratio

    #original image
    #cv2.imshow("Original Image", image)
    #cv2.waitKey(0) #waits till a key is pressed
    # RGB to Gray scale conversion
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # Remove Noise, Sharpness (Averaging)
    gray = cv2.bilateralFilter(gray, 11, 17, 17)
    
    # Find Edges of the grayscale image - Canny Edge Detection 
    edged = cv2.Canny(gray, 170, 200)
    
    # Find number contours based on Edges
    cnts, new  = cv2.findContours(edged.copy(), cv2.RETR_LIST, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create copy of original image to draw all contours
    #we create a copy so as to not destroy the original image
    img1 = image.copy()
    cv2.drawContours(img1, cnts, -1, (0,255,0), 3)
    
    #sort contours based on their area keeping minimum required area as '30' (anything smaller than this will not be considered)
    #max to min sorting
    cnts=sorted(cnts, key = cv2.contourArea, reverse = True)[:30]
    NumberPlateCnt = None 

    # Top 30 Contours
    img2 = image.copy()
    cv2.drawContours(img2, cnts, -1, (0,255,0), 3)

    # loop over each of thr top 30 contours 
    #find the best possible approximate contour of number plate
    
    for c in cnts:
            peri = cv2.arcLength(c, True) #find perimeter
            approx = cv2.approxPolyDP(c, 0.02 * peri, True) #no. of edges for each contour
            if len(approx) == 4:  # Select the contour with 4 corners
                NumberPlateCnt = approx #This is our approx Number Plate Contour

                #crop according to coordinates - our ROI
                x, y, w, h = cv2.boundingRect(c) #This will find out co-ord for plate
                new_img = gray[y:y + h, x:x + w] #crop the grayscale image
                #cv2.imwrite('BoundedBox/' + str(idx) + '.jpg', new_img) #Store new image
                #idx+=1

                break

    # Drawing the selected contour on the original image
     #
    
    if NumberPlateCnt is None:
        continue;
    cv2.drawContours(image, [NumberPlateCnt], -1, (0,255,0), 3)
    #cv2.imshow("Final Image With Number Plate Detected", image)
    #cv2.waitKey(0)
    
    cv2.imwrite('BoundedBox/' + str(idx) + '.jpg', new_img) #Store new image
    #cv2.imshow("Cropped Image ", new_img)
    #cv2.waitKey(0)
    #print(idx)
    #idx = idx + 1
    
    # Resize